In [351]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt 
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import cv2
import datetime
import xgboost as xgb

In [352]:
train=pd.read_csv('~/Downloads/Train.csv')
test=pd.read_csv('~/Downloads/Test.csv')

In [353]:
train.shape

(8523, 12)

In [354]:
train.loc[ train['Outlet_Establishment_Year'] <= 1995, 'Outlet_Establishment_Year'] = 0
train.loc[(train['Outlet_Establishment_Year'] > 1995) & (train['Outlet_Establishment_Year'] <= 2005), 'Outlet_Establishment_Year'] = 1
train.loc[train['Outlet_Establishment_Year'] > 2005, 'Outlet_Establishment_Year'] = 2
 
test.loc[ train['Outlet_Establishment_Year'] <= 1995, 'Outlet_Establishment_Year'] = 0
test.loc[(train['Outlet_Establishment_Year'] > 1995) & (test['Outlet_Establishment_Year'] <= 2005), 'Outlet_Establishment_Year'] = 1
test.loc[train['Outlet_Establishment_Year'] > 2005, 'Outlet_Establishment_Year'] = 2    

In [355]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,0,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,0,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,0,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,1,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2,NaN,Tier 2,Supermarket Type1,4710.5350


In [356]:
# train.loc[ train['Item_Fat_Content'] == 'Low Fat'|train['Item_Fat_Content'] == 'LF'|train['Item_Fat_Content'] == 'low fat', 'Item_Fat_Content'] = 0
# train.loc[ train['Item_Fat_Content'] == 'Regular', 'Item_Fat_Content'] = 1
train['Item_Fat_Content'] = train['Item_Fat_Content'].map( {'Low Fat': 0, 'low fat': 0, 'LF': 0, 'Regular':1,'reg':1} ).astype(int)

In [357]:
test['Item_Fat_Content'] = test['Item_Fat_Content'].map( {'Low Fat': 0, 'low fat': 0, 'LF': 0, 'Regular':1,'reg':1} ).astype(int)

In [358]:
y=train['Item_Outlet_Sales']

sub_ids = test[['Item_Identifier','Outlet_Identifier']]
test = test.drop(['Item_Identifier','Outlet_Identifier'], axis=1)
train = train.drop(['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'], axis=1)




In [359]:
train.dtypes

Item_Weight                  float64
Item_Fat_Content               int64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
dtype: object

In [360]:
train['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [361]:
train['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [362]:
test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0], inplace=True)
train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0], inplace=True)

In [363]:
obj_cols = [x for x in train.columns if train[x].dtype == 'object']

In [364]:
encoder=LabelEncoder()
for x in obj_cols:
    encoder.fit(train[x])
    train[x] = encoder.transform(train[x])
    test[x] = encoder.transform(test[x])

In [365]:
train.head(10)

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.300,0,0.016047,4,249.8092,1,1,0,1
1,5.920,1,0.019278,14,48.2692,2,1,2,2
2,17.500,0,0.016760,10,141.6180,1,1,0,1
3,19.200,1,0.000000,6,182.0950,1,1,2,0
4,8.930,0,0.000000,9,53.8614,0,0,2,1
5,10.395,1,0.000000,0,51.4008,2,1,2,2
6,13.650,1,0.012741,13,57.6588,0,0,2,1
7,NaN,0,0.127470,13,107.7622,0,1,2,3
8,16.200,1,0.016687,5,96.9726,1,1,1,1
9,19.200,1,0.094450,5,187.8214,2,1,1,1


In [366]:
test['Item_Weight'].fillna(test['Item_Weight'].median(), inplace=True)
train['Item_Weight'].fillna(train['Item_Weight'].median(), inplace=True)

In [367]:
X_train,x_test,Y_train,y_test=train_test_split(train,y,test_size=0.2,random_state=4)

In [372]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

# forest_reg = GradientBoostingRegressor(random_state=6,n_estimators=130)
forest_reg=xgb.XGBRegressor(max_depth=3, n_estimators=100, learning_rate=0.05)
# forest_reg = AdaBoostRegressor()

scores = cross_val_score(forest_reg, train, y, scoring='r2', cv=5)
print(scores)


print('mean r2:',np.mean(scores))

[ 0.60780086  0.58617868  0.57887329  0.60786412  0.60817102]
mean r2: 0.597777593782


In [376]:
#Testing Purpose, find for best suit classifier

# forest_reg = GradientBoostingRegressor(max_depth=4, n_estimators=110, learning_rate=0.03).fit(X_train, Y_train)

# forest_reg = xgb.XGBRegressor(max_depth=3, n_estimators=100, learning_rate=0.05).fit(X_train, Y_train) 

# forest_reg = RandomForestRegressor(max_depth=3, n_estimators=120).fit(X_train, Y_train)
forest_reg = MLPRegressor(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=200, random_state=1).fit(X_train, Y_train) 

preds = forest_reg.predict(x_test)

mean_squared_error(y_test, preds) 

1371836.2393126336

In [371]:
from IPython.display import FileLink


forest_reg1 = xgb.XGBRegressor(max_depth=4, n_estimators=110, learning_rate=0.03).fit(train, y)
forest_reg2 = GradientBoostingRegressor(max_depth=3, n_estimators=140, learning_rate=0.03).fit(train, y)
forest_reg3 = RandomForestRegressor(max_depth=3, n_estimators=120).fit(train,y)

pred_1 = forest_reg1.predict(test)
pred_2 = forest_reg2.predict(test)
pred_3 = forest_reg3.predict(test)

preds=(pred_1+pred_2+pred_3)/3


sub = pd.DataFrame({'Item_Identifier': sub_ids['Item_Identifier'], 'Item_Outlet_Sales': preds,'Outlet_Identifier': sub_ids['Outlet_Identifier'],})
filename = 'xbmSales1.csv'

sub.to_csv(filename, index=False)


FileLink(filename)  


/home/paras/paras/xbmSales1.csv